<a href="https://colab.research.google.com/github/ShawnLiu119/Segmentation_Embedding_DL/blob/main/Customer_Embedding_EventSequence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Customer Embedding - Based on Event**

## Key Concept
**Event**: customer event sequence could be product browsing history on website, clicks, in store purchase, transactions </br>
**Doc2Vec**: vectorize the document: Doc2Vec is a neural network-based approach that learns the distributed representation of documents. It is an unsupervised learning technique that maps each document to a fixed-length vector in a high-dimensional space

## Use Case
1. **segmentation** capture behavioral semantics and use these embeddings for audience analysis (e.g. clustering)
2. **recommendation engine** input for the downstream personalization models

## Logical thoughts
Each customer is viewed as a document, orders as sentences, and products as words. Each customer could place multiple orders, while one order could include multiple products

## MLOPs consideration
 The dataset size and compute resources need to be carefully selected because Doc2Vec and subsequent clustering are computationally heavy processes.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from tqdm import tqdm_notebook as tqdm #show progress bar
import numpy as np
import os
from sklearn.manifold import TSNE #visualize high-dimensional data by converting similarity to join probabilities

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn import preprocessing

pd.options.display.max_rows = 20
%matplotlib inline

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 500)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import glob

import multiprocessing as mp
print('Number of CPU cores:', mp.cpu_count())